In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from typing import Any, Optional

import evals.core.main as core
import evals.scr_and_tpp.main as scr_and_tpp
import evals.sparse_probing.main as sparse_probing
import sae_bench_utils.general_utils as general_utils
import custom_saes.custom_sae_config as custom_sae_config
import custom_saes.vanilla_sae as vanilla_sae
from sae_bench_utils.sae_selection_utils import get_saes_from_regex
import custom_saes.run_all_evals_custom_saes as run_all_evals_custom_saes

RANDOM_SEED = 42

output_folders = {
    "absorption": "eval_results/absorption",
    "autointerp": "eval_results/autointerp",
    "core": "eval_results/core",
    "scr": "eval_results/scr",
    "tpp": "eval_results/tpp",
    "sparse_probing": "eval_results/sparse_probing",
    "unlearning": "eval_results/unlearning",
}

# Note: Unlearning is not recommended for models with < 2B parameters and we recommend an instruct tuned model
# Unlearning will also require requesting permission for the WMDP dataset (see unlearning/README.md)
# Absorption not recommended for models < 2B parameters
# asyncio doesn't like notebooks, so autointerp must be ran using a python script

# Select your eval types here.
eval_types = [
    "absorption",
    # "autointerp",
    "core",
    "scr",
    "tpp",
    "sparse_probing",
    # "unlearning",
]

if "autointerp" in eval_types:
    raise ValueError("autointerp must be ran using a python script")

device = general_utils.setup_environment()

model_name = "pythia-70m-deduped"
llm_batch_size = 512
dtype = "float32"


# If evaluating multiple SAEs on the same layer, set save_activations to True
# This will require at least 100GB of disk space
save_activations = False

/gpfs/radev/home/tl784/.conda/envs/ai_scientist/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


This cell loads your custom SAEs. If you just want to use existing SAE Lens SAEs, comment it out.

In [3]:
repo_id = "canrager/lm_sae"
baseline_filename = "pythia70m_sweep_standard_ctx128_0712/resid_post_layer_4/trainer_8/ae.pt"
hook_layer = 4
hook_name = f"blocks.{hook_layer}.hook_resid_post"

sae = vanilla_sae.load_vanilla_sae(repo_id, baseline_filename, hook_layer)
sae = sae.to(device, dtype=general_utils.str_to_dtype(dtype))

print(f"sae dtype: {sae.dtype}, device: {sae.device}")

d_sae, d_in = sae.W_dec.data.shape

assert d_sae >= d_in

print(f"d_in: {d_in}, d_sae: {d_sae}")

/gpfs/radev/project/lafferty/tl784/AI-Scientist/custom_saes/vanilla_sae.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pt_params = torch.load(path_to_params, map_locat

Original keys in state_dict: odict_keys(['bias', 'encoder.weight', 'encoder.bias', 'decoder.weight'])
Renamed keys in state_dict: dict_keys(['b_dec', 'W_enc', 'b_enc', 'W_dec'])
d_in: 512
d_sae: 4096
sae dtype: torch.float32, device: cuda
d_in: 512, d_sae: 4096


In our sae object we need to have a CustomSAEConfig. This contains some information which is used by the evals (hook_name, hook_layer, model_name, d_sae, etc). In addition, it contains information that is used by our plotting functions, like number of training tokens and architecture. For example, we should have the sae.cfg.architecture defined if we want to plot multiple SAE architectures.

In [4]:
sae.cfg = custom_sae_config.CustomSAEConfig(
    model_name, d_in=d_in, d_sae=d_sae, hook_name=hook_name, hook_layer=hook_layer
)

# Core evals require us to specify the dtype. This must be a string that can be converted to a torch dtype using general_utils.str_to_dtype.
sae.cfg.dtype = dtype


# The following contains our current defined SAE types and the shapes to plot for each. Add your custom SAE as new_sae_key
new_sae_key = "vanilla"
trainer_markers = {
    "standard": "o",
    "jumprelu": "X",
    "topk": "^",
    "p_anneal": "*",
    "gated": "d",
    new_sae_key: "s",  # New SAE
}

trainer_colors = {
    "standard": "blue",
    "jumprelu": "orange",
    "topk": "green",
    "p_anneal": "red",
    "gated": "purple",
    new_sae_key: "black",  # New SAE
}

sae.cfg.architecture = new_sae_key
sae.cfg.training_tokens = 200_000_000

`selected_saes` is a list of tuples of (unique_sae_id, sae object) OR (sae lens release, sae lens id). If it is a list of custom sae objects, then memory size will increase with the length of the list. This is especially important if the SAEs are large. If memory is a concern, I recommend calling the `run_eval()` function multiple times with lists of length 1, each list containing a new sae object.

In [5]:
# Note: the custom_sae_id should be unique, as it is used for the intermediate results and final results file names

unique_custom_sae_id = baseline_filename.replace("/", "_").replace(".", "_")
print(f"sae_id: {unique_custom_sae_id}")

# list of tuple of (sae_id, sae object)
custom_saes = [(unique_custom_sae_id, sae)]

sae_id: pythia70m_sweep_standard_ctx128_0712_resid_post_layer_4_trainer_8_ae_pt


Select your baseline SAEs here. Refer to `sae_regex_selection.ipynb` for more regex patterns. We are going to get a topk SAE from the same layer.

In [6]:
sae_regex_pattern = r"(sae_bench_pythia70m_sweep_topk_ctx128_0730).*"
sae_block_pattern = r".*blocks\.([4])\.hook_resid_post__trainer_(8)$"

baseline_saes = get_saes_from_regex(sae_regex_pattern, sae_block_pattern)
print(f"baseline_saes: {baseline_saes}")
baseline_sae_id = f"{baseline_saes[0][0]}_{baseline_saes[0][1]}".replace(".", "_")
print(f"baseline_sae_id: {baseline_sae_id}")

selected_saes = custom_saes + baseline_saes

100%|██████████| 57/57 [00:00<00:00, 34187.81it/s]

baseline_saes: [('sae_bench_pythia70m_sweep_topk_ctx128_0730', 'blocks.4.hook_resid_post__trainer_8')]
baseline_sae_id: sae_bench_pythia70m_sweep_topk_ctx128_0730_blocks_4_hook_resid_post__trainer_8


Run time for the next 2 functions is approximately 2 minutes on an RTX 3090.

In [7]:
# Note: We typically run with n_eval_sparsity_variance_batches=2000, but I have reduced it here for a faster run

_ = core.multiple_evals(
    selected_saes=selected_saes,
    n_eval_reconstruction_batches=200,
    n_eval_sparsity_variance_batches=200,
    eval_batch_size_prompts=32,
    compute_featurewise_density_statistics=False,
    compute_featurewise_weight_based_metrics=False,
    exclude_special_tokens_from_reconstruction=True,
    dataset="Skylion007/openwebtext",
    context_size=128,
    output_folder="eval_results/core",
    verbose=True,
    dtype=dtype,
)

Using device: cuda


Running SAE evaluation on all selected SAEs:   0%|          | 0/2 [00:00<?, ?it/s]/gpfs/radev/home/tl784/.conda/envs/ai_scientist/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model pythia-70m-deduped into HookedTransformer


/gpfs/radev/home/tl784/.conda/envs/ai_scientist/lib/python3.11/site-packages/sae_lens/training/activations_store.py:285: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(
Sparsity and Variance Batches: 100%|██████████| 200/200 [00:04<00:00, 46.23it/s]


Saved evaluation results to: eval_results/core/pythia70m_sweep_standard_ctx128_0712_resid_post_layer_4_trainer_8_ae_pt_custom_sae_eval_results.json


Running SAE evaluation on all selected SAEs:   0%|          | 0/2 [00:21<?, ?it/s]

asdict clause runned.
eval_results[-1] = defaultdict(<function nested_dict at 0x15151f96c9a0>, {'unique_id': 'pythia70m_sweep_standard_ctx128_0712_resid_post_layer_4_trainer_8_ae_pt_custom_sae', 'sae_set': 'pythia70m_sweep_standard_ctx128_0712_resid_post_layer_4_trainer_8_ae_pt', 'sae_id': 'custom_sae', 'eval_cfg': CoreEvalConfig(model_name='pythia-70m-deduped', llm_dtype='float32', batch_size_prompts=32, n_eval_reconstruction_batches=200, n_eval_sparsity_variance_batches=200, dataset='Skylion007/openwebtext', context_size=128, compute_kl=True, compute_ce_loss=True, compute_l2_norms=True, compute_sparsity_metrics=True, compute_variance_metrics=True, compute_featurewise_density_statistics=False, compute_featurewise_weight_based_metrics=False, exclude_special_tokens_from_reconstruction=True, verbose=False), 'metrics': {'model_behavior_preservation': {'kl_div_score': nan, 'kl_div_with_ablation': nan, 'kl_div_with_sae': nan}, 'model_performance_preservation': {'ce_loss_score': 0.9223507948

TypeError: Object of type CoreEvalConfig is not JSON serializable

In [ ]:
# We do a subset of the sparse probing datasets here for shorter runtime
dataset_names = ["LabHC/bias_in_bios_class_set1"]

# TODO: Add a verbose flag

_ = sparse_probing.run_eval(
    sparse_probing.SparseProbingEvalConfig(
        model_name=model_name,
        random_seed=RANDOM_SEED,
        llm_batch_size=llm_batch_size,
        llm_dtype=dtype,
        dataset_names=dataset_names,
    ),
    selected_saes,
    device,
    "eval_results/sparse_probing",
    force_rerun=False,
    clean_up_activations=True,
    save_activations=save_activations,
)


In [ ]:
import matplotlib.pyplot as plt
import json
import torch
import pickle
from typing import Optional
from matplotlib.colors import Normalize
import numpy as np
import os

import sae_bench_utils.graphing_utils as graphing_utils

from sae_bench_utils.sae_selection_utils import select_saes_multiple_patterns

In [ ]:
eval_path = output_folders["sparse_probing"]

core_results_path = output_folders["core"]
image_path = "./images"

if not os.path.exists(image_path):
    os.makedirs(image_path)

In [ ]:
custom_sae_ids = []

for sae_id, sae in custom_saes:
    custom_sae_ids.append((sae_id, "custom_sae"))

sae_lens_ids = []

for sae_id, sae_release in baseline_saes:
    sae_lens_ids.append((sae_id, sae_release))

graphing_sae_ids = custom_sae_ids + sae_lens_ids

Here we can view the raw results, and we see that both SAEs significantly outperform the residual stream baseline.

In [ ]:
raw_results_dict = graphing_utils.get_results_dict(graphing_sae_ids, eval_path, core_results_path)

print(raw_results_dict.keys())

In [ ]:
custom_sae_id = f"{custom_sae_ids[0][0]}_{custom_sae_ids[0][1]}".replace(".", "_")
baseline_sae_id = f"{sae_lens_ids[0][0]}_{sae_lens_ids[0][1]}"


baseline_filename = f"{sae_lens_ids[0][0]}_{sae_lens_ids[0][1]}_eval_results.json".replace("/", "_")
baseline_filepath = os.path.join(eval_path, baseline_filename)

with open(baseline_filepath, "r") as f:
    baseline_sae_eval_results = json.load(f)

custom_filename = f"{custom_sae_ids[0][0]}_{custom_sae_ids[0][1]}_eval_results.json".replace(
    "/", "_"
)
custom_filepath = os.path.join(eval_path, custom_filename)

with open(custom_filepath, "r") as f:
    custom_sae_eval_results = json.load(f)

k = 1

print(baseline_sae_eval_results.keys())

print(
    f"Baseline SAE top {k} accuracy was:",
    baseline_sae_eval_results["eval_result_metrics"]["sae"][f"sae_top_{k}_test_accuracy"],
)
print(
    f"Custom SAE top {k} accuracy was:",
    custom_sae_eval_results["eval_result_metrics"]["sae"][f"sae_top_{k}_test_accuracy"],
)
print(
    f"LLM top {k} accuracy was:",
    baseline_sae_eval_results["eval_result_metrics"]["llm"][f"llm_top_{k}_test_accuracy"],
)

We can also plot the metrics, plotting L0 vs Custom Metric or L0 vs Loss Recovered vs Custom metric. We can have different shapes for the SAE type or dictionary size.

In [ ]:
image_base_name = os.path.join(image_path, "sparse_probing")

graphing_utils.plot_results(
    graphing_sae_ids,
    eval_path,
    core_results_path,
    image_base_name,
    k,
    trainer_markers=trainer_markers,
    trainer_colors=trainer_colors,
)

Now we will run all of the evals, and create more plots. Be warned - this takes around an hour. Note that a significant amount of the costs are one time per layer - for example, with absorption, we have to train a bunch of probes on a given layer. So, if we have multiple SAEs per layer the cost should be much less than 30 minutes per SAE. In addition, to save disk space usage we currently are not saving activations for reuse by multiple SAEs.

Additionally, we can make this faster by evaluating on a subset of the datasets. Sparse probing and Spurious Correlation Removal both have approximately 8 datasets each. We could lower the time by using fewer datasets at the cost of not having as strong of a signal.

In [ ]:
_ = run_all_evals_custom_saes.run_evals(
    model_name,
    selected_saes,
    llm_batch_size,
    dtype,
    device,
    eval_types,
    api_key=None,
    force_rerun=False,
    save_activations=save_activations,
)

In [ ]:
for eval_type in eval_types:
    graphing_utils.plot_results(
        graphing_sae_ids,
        output_folders[eval_type],
        core_results_path,
        image_base_name,
        k=10,
        trainer_markers=trainer_markers,
        trainer_colors=trainer_colors,
    )